In [1]:
import anndata
import pandas as pd
import scimap as sm
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import numpy as np
import seaborn as sns
warnings.simplefilter(action='ignore', category=FutureWarning)

Running SCIMAP  2.2.11


This script performs NEP analysis on the MI dataset with scimap using scimap https://github.com/labsyspharm/scimap

In [4]:
path_to_csv = './../../../../../MI_heart_paper/data/cell_table_final.csv'
obs = pd.read_csv(path_to_csv)
output_path = './../../../Comparison/20250218_results_MI/MI_scimap_knn5.csv'
label = 'scimap_knn5'

In [5]:
# exclude the specified cell types in obs
ignore_cell_types = ['exclude']
obs = obs[~obs['final_cell_type'].isin(ignore_cell_types)]

In [6]:
# add marker files to it, as SpatialLDA needs them
obs['D'] = np.random.randint(1, 101, size=len(obs))
obs['E'] = np.random.randint(1, 101, size=len(obs))

# Ensure obs has a string-based index
obs.index = obs.index.astype(str)  # Convert index to string


# Load dataframe into anndata object
# the data matrix 
X = obs[['D', 'E']]
X = X.values
adata = anndata.AnnData(X)
adata.obs = obs
adata

AnnData object with n_obs × n_vars = 563180 × 2
    obs: 'fov', 'label', 'cell_size', 'X_centroid', 'Y_centroid', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'cell_meta_cluster', 'timepoint', 'region', 'region_name', 'refined_cell_type', 'final_cell_type', 'endocardial_annotation', 'exclude_annotation', 'artefact', 'distance_from_lumen', 'lumen_bin', 'size_filter', 'D', 'E'

In [7]:
# Define the spatial interaction analysis
spatial_params = {
    'adata': adata,
    'x_coordinate': 'X_centroid',
    'y_coordinate': 'Y_centroid',
    'phenotype': 'final_cell_type',
    'method': 'knn',
    'knn': 5,
    'permutation': 300,
    'imageid': 'fov',
    'subset': None,
    'pval_method': 'zscore',
    'verbose': True,
    'label': label
}
    
# Run spatial interaction analysis with the conditional parameter
spatial = sm.tl.spatial_interaction(**spatial_params)


Processing Image: ['24h_86']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['24h_83']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['Control_13']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['Control_12']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['4h_97']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['4h_96']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to ne

In [8]:
#only include columns with zscore and replace the zscore_ with empty string
spatial = adata.uns[label]
spatial = pd.DataFrame(spatial)

spatial['interaction'] = spatial['phenotype'] + '_' + spatial['neighbour_phenotype']
# remove the two columns
spatial = spatial.drop(columns=['phenotype', 'neighbour_phenotype'])
spatial = spatial.set_index('interaction')

# filter for columns that do not contain the word pvalue
spatial = spatial.filter(regex='^(?!.*pvalue).*$', axis=1)
spatial.T.to_csv('./../../../Comparison/20250218_results_MI/MI_scimap_knn5.csv')
spatial

,24h_86,24h_83,Control_13,Control_12,4h_97,4h_96,48h_79,48h_76,Control_14
interaction,,,,,,,,,
Cardiomyocytes_Cardiomyocytes,172.597395,162.847521,-7.760266,30.373068,100.248900,145.881880,97.291837,147.867932,68.680865
Cardiomyocytes_Cardiomyocytes Ankrd1+,-174.099000,-191.817166,-4.839652,-20.638595,-112.200721,-159.336431,-128.758182,-113.725969,-7.268301
Cardiomyocytes_Endocardial cells,-19.719067,-11.742726,-18.919678,-19.120221,-16.132206,-5.345564,-5.365580,6.000319,-31.544313
Cardiomyocytes_Endothelial cells,32.502566,78.023902,33.773093,-4.528563,10.744829,16.897762,86.056614,46.308083,-18.207881
Cardiomyocytes_Fibroblasts,0.409865,16.543899,-16.036986,-26.307792,-12.787311,-1.956574,4.291058,0.195236,-33.620443
...,...,...,...,...,...,...,...,...,...
Smooth muscle cells_Macrophages Trem2-,-0.941953,-2.238652,-2.815764,3.241867,-0.548231,-1.604428,-0.236273,-1.933083,-1.185979
Smooth muscle cells_Mono / Macros Ccr2+,0.281839,-3.826066,-1.267049,2.109042,0.113609,2.389540,-10.485763,-7.695223,0.460048
Smooth muscle cells_Neutrophils,-5.219159,-6.121588,-0.670901,-0.986755,-7.434309,-3.767523,-5.380420,-5.645070,-0.896263
